## General Import

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import gc
import time
import seaborn as sns; sns.set()

import os 
# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
#pd.options.display.mpl_style = 'default'
from datetime import datetime
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
from dateutil import tz
import geojson
import geopandas as gpd
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint
%matplotlib inline
import matplotlib.pyplot as plt
from geopandas.tools import sjoin

import matplotlib.cm as cm

import folium
import shapely.geometry

from branca.colormap import  linear
import json
import branca.colormap as cm

import matplotlib.colors as colors
%matplotlib inline

import networkx as nx

import pickle

import osmnx as ox
ox.config(log_console=True, use_cache=True)
ox.__version__

There is 1 csv file in the current version of the dataset:


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Let's check 1st file: /kaggle/input/BXL_timeseries_kaggle.csv

#### note: install OSMNX package (enable internet connection)

* *!pip install osmnx*

In [ ]:
DF_NODES_FEATURES = pd.read_csv('/kaggle/input/obu-data-preprocessing/Flow_BXL_street_15min.csv')
STREETS = list(DF_NODES_FEATURES.columns[1:])
STREETS = [int(float(street)) for street in STREETS]
STREETS = ['datetime'] + STREETS
DF_NODES_FEATURES.columns = STREETS
print(DF_NODES_FEATURES.shape)

# Download Streets Network from OSMNX

In [ ]:
G = ox.graph_from_place('Brussels Capital Region, Belgium', network_type='drive', custom_filter='["highway"~"motorway|motorway_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link|trunk_link"]')
G = ox.project_graph(G)

nodes, edges = ox.graph_to_gdfs(G)

print('edges '+str(edges.shape))
print('nodes '+str(nodes.shape))

In [ ]:
fig, ax = ox.plot_graph(G, bgcolor='k', node_size=10, 
                        node_color='#999999', node_edgecolor='none', node_zorder=1,
                        edge_color='#555555', edge_linewidth=0.5, edge_alpha=1)



In [ ]:
nx.to_numpy_matrix(G).shape

In [ ]:
edges['Ring'] = edges.geometry.is_ring
edges['centroids'] = edges.geometry.centroid

first_points = []
end_points = []

for row in range(len(edges)):

    first, last = Point(edges.iloc[row].geometry.coords[0]), Point(edges.iloc[row].geometry.coords[-1])
    
    first_points.append(first)
    end_points.append(last)
    
edges['first'] =  first_points  
edges['last'] =  end_points  
    
edges["lat_centroid"] = edges.centroids.map(lambda p: p.x)
edges["lon_centroid"] = edges.centroids.map(lambda p: p.y)
edges['point'] = edges.apply(lambda x: (x['lon_centroid'], x['lat_centroid']), axis=1)

In [ ]:
# edges[edges['osmid']==139734095]
edges

In [ ]:
sample = edges.reset_index()
sample = sample[['index','u','v','first','last','centroids','point', 'geometry','length']]
sample['node'] = sample['index'].astype(str)+'_'+sample['u'].astype(str)+'_'+sample['v'].astype(str)
sample = sample[['node','u','v','first','last','centroids','point', 'geometry','length']]
sample.shape

In [ ]:
sigma = np.std(sample['length'])

In [ ]:
# sep = ' - '
# sample['node'] = sample.node.apply(lambda r: r.split(sep, 1)[-1])
# sample

In [ ]:
from tqdm import tqdm

u_list = []
v_list = []

weight_list = []

for i in tqdm(range(len(sample))):
    
    df = sample.iloc[i:,:]
        
    for k in range(1,len(df)):
        

        if (df.iloc[0]['first'] == df.iloc[k]['last']) and (df.iloc[0]['last'] == df.iloc[k]['first']): # and (df.iloc[0]['centroids'] != df.iloc[k]['centroids']) : #

            u_list.append(str(df.iloc[k]['node']))
            v_list.append(str(df.iloc[0]['node']))
            
#             weight_list.append(np.exp(1/((df.iloc[k]['length']/2 + df.iloc[0]['length']/2)**2/sigma)))
            
            u_list.append(str(df.iloc[0]['node']))
            v_list.append(str(df.iloc[k]['node']))
            
#             weight_list.append(np.exp(1/((df.iloc[k]['length']/2 + df.iloc[0]['length']/2)**2/sigma)))

            
        elif (df.iloc[0]['first'] == df.iloc[k]['last']) and (df.iloc[0]['last'] != df.iloc[k]['first']): #


            u_list.append(str(df.iloc[k]['node']))
            v_list.append(str(df.iloc[0]['node']))
            
#             weight_list.append(np.exp(1/((df.iloc[k]['length']/2 + df.iloc[0]['length']/2)**2/sigma)))
            
            
        elif (df.iloc[0]['first'] != df.iloc[k]['last']) and (df.iloc[0]['last'] == df.iloc[k]['first']): #


            u_list.append(str(df.iloc[0]['node']))
            v_list.append(str(df.iloc[k]['node']))
            
#             weight_list.append(np.exp(1/((df.iloc[k]['length']/2 + df.iloc[0]['length']/2)**2/sigma)))
            
                          
        else:

            u_list.append('no consecutive')
            v_list.append('no consecutive')
            
#             weight_list.append(np.exp(1/((df.iloc[k]['length']/2 + df.iloc[0]['length']/2)**2/sigma)))

In [ ]:
# print('check if columns dataframe is equal to edges in osmnx')
# print('')

# STREETS = list(DF_NODES_FEATURES.columns[1:]) #without datetime
# EDGES = list(edges.index)

# print('are the two lists of same length?')
# print(len(STREETS)==len(EDGES))

In [ ]:
# ADD_STREETS = list(set(EDGES) - set(STREETS))

# rows_streets = len(DF_NODES_FEATURES)
# columns_streets = len(list(set(EDGES) - set(STREETS)))
# zeros_array = np.zeros(shape=(rows_streets, columns_streets))
    
# add_streets = pd.DataFrame(zeros_array, columns=ADD_STREETS)

In [ ]:
# nodes_features = pd.concat([DF_NODES_FEATURES, add_streets], axis=1)

# nodes_features = nodes_features[list(edges.index)]

# nodes_features['datetime'] = DF_NODES_FEATURES['datetime']

# print(nodes_features.shape)

# nodes_features.to_csv('FEATURES.csv')

In [ ]:
# FINAL_STREETS = list(set(STREETS) & set(EDGES))

# edges = edges[edges['index'].isin(FINAL_STREETS)]
# print(edges.shape)
# DF_NODES_FEATURES = DF_NODES_FEATURES[DF_NODES_FEATURES.columns.intersection(FINAL_STREETS)]
# print(DF_NODES_FEATURES.shape)
# DF_NODES_FEATURES.to_csv('FEATURES.csv')

In [ ]:
# edges['centroid'] = edges['geometry'].centroid
# edges["lat"] = edges.centroid.map(lambda p: p.x)
# edges["lon"] = edges.centroid.map(lambda p: p.y)#.astype(int)
# edges['point'] = edges.apply(lambda x: (x['lon'], x['lat']), axis=1)

# edges.head()

In [ ]:
# sample = edges[['index','u','v','maxspeed','length','point']]
# sample['node'] = sample['index'].astype(str)+'_'+sample['u'].astype(str)+'_'+sample['v'].astype(str)
# sample = sample[['node','u','v','point']]
# sample.shape

In [ ]:
# sep = ' - '
# sample['node'] = sample.node.apply(lambda r: r.split(sep, 1)[-1])
# sample

In [ ]:
# from tqdm import tqdm

# u_list = []
# v_list = []

# for i in tqdm(range(len(sample))):
    
#     df = sample.iloc[i:,:]
        
#     for k in range(1,len(df)):
        

#         if (df.iloc[0]['u'] == df.iloc[k]['v']) and (df.iloc[0]['v'] == df.iloc[k]['u']): #

# #             u_list.append(str(df.iloc[k]['node']))
# #             v_list.append(str(df.iloc[0]['node']))
            
# #             u_list.append(str(df.iloc[0]['node']))
# #             v_list.append(str(df.iloc[k]['node']))

#             u_list.append('no consecutive')
#             v_list.append('no consecutive')

            

        
#         elif (df.iloc[0]['u'] == df.iloc[k]['v']) and (df.iloc[0]['v'] != df.iloc[k]['u']): #


#             u_list.append(str(df.iloc[k]['node']))
#             v_list.append(str(df.iloc[0]['node']))
            
            
#         elif (df.iloc[0]['u'] != df.iloc[k]['v']) and (df.iloc[0]['v'] == df.iloc[k]['u']): #


#             u_list.append(str(df.iloc[0]['node']))
#             v_list.append(str(df.iloc[k]['node']))
            
                          
#         else:

#             u_list.append('no consecutive')
#             v_list.append('no consecutive')

            


In [ ]:
df_u = pd.DataFrame (u_list,columns=['u'])
df_v = pd.DataFrame (v_list,columns=['v'])
# df_weight = pd.DataFrame (weight_list,columns=['weight'])

# df_union = pd.concat([df_u ,df_v, df_weight], axis=1)
df_union = pd.concat([df_u ,df_v], axis=1)
df_union = df_union[df_union.u != 'no consecutive']

# df_union_dupl = df_union[df_union['u'] != df_union['v']]

df_union.shape

In [ ]:
df = df_union[['u','v']]
G = nx.from_pandas_edgelist(df,'u','v')#, create_using = nx.MultiDiGraph())
#nx.from_pandas_edgelist(df,'u','v', edge_attr='weight', create_using = nx.MultiDiGraph()) #nx.DiGraph())

plt.figure(figsize=(50, 30))
nx.draw(G)


In [ ]:
components = [G.subgraph(c).copy() for c in nx.connected_components(G)] #[G.subgraph(c).copy() for c in nx.weakly_connected_components(G)] 

for idx,g in enumerate(components,start=1):
    print(f"subgraph {idx}")
    
G_main = components[0]

plt.figure(figsize=(50, 30))
nx.draw(G_main)

In [ ]:
len(list(G_main.nodes))

In [ ]:
sample = sample[sample['node'].isin(list(G_main.nodes))] 
streets_graph = list(sample.geometry)
edges = edges[edges['geometry'].isin(streets_graph)]
edges.shape

In [ ]:
node_position = {}

for node in sample.values: # Loop through the list, one row at a time
    node_position[node[0]] = node[-3][::-1]

nx.set_node_attributes(G_main, node_position, 'pos')

# --------------------------------------


In [ ]:
# durations = [i['weight'] for i in dict(G_main.edges).values()]

In [ ]:
pos = nx.get_node_attributes(G_main,'pos')

G_main.add_nodes_from(pos.keys())

plt.figure(figsize=(30, 20))
nx.draw(G_main, pos=pos, node_size=100)
plt.show()

In [ ]:
# num_nodes = len(G_main)
# order = sorted(list(G_main.nodes()))
# + np.diag(np.ones(num_nodes))
    
adj_mx = nx.to_numpy_matrix(G_main)
adj_mx.shape
# plt.imshow(adj_mx, cmap='hot', interpolation='nearest')
# plt.show()


In [ ]:
plt.figure(figsize=(30, 20))
Node = nx.draw_networkx_nodes(G_main, pos, node_size=100, node_color="blue", labels=True)
Edges = nx.draw_networkx_edges(
    G_main,
    pos,
    node_size=100,
    arrowstyle="->",
    arrowsize=20
    , labels=True)


plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
edges['geometry'].plot(figsize=(20, 10), color='gold')
Node = nx.draw_networkx_nodes(G_main, pos, node_size=20, node_color="blue", labels=True)
Edges = nx.draw_networkx_edges(
    G_main,
    pos,
    node_size=20,
    arrowstyle="->",
    arrowsize=20,
    labels=True)


plt.show()

In [ ]:
print('check if columns dataframe is equal to edges in osmnx')
print('')

STREETS = list(DF_NODES_FEATURES.columns[1:]) #without datetime
EDGES = list(edges.index)

print('are the two lists of same length?')
print(len(STREETS)==len(EDGES))

In [ ]:
ADD_STREETS = list(set(EDGES) - set(STREETS))

rows_streets = len(DF_NODES_FEATURES)
columns_streets = len(list(set(EDGES) - set(STREETS)))
zeros_array = np.zeros(shape=(rows_streets, columns_streets))
    
add_streets = pd.DataFrame(zeros_array, columns=ADD_STREETS)

In [ ]:
nodes_features = pd.concat([DF_NODES_FEATURES, add_streets], axis=1)

nodes_features = nodes_features[list(edges.index)]

nodes_features['datetime'] = DF_NODES_FEATURES['datetime']

print(nodes_features.shape)

nodes_features.to_csv('FEATURES.csv')

In [ ]:
# Saving the objects:
with open('graph_adj.pkl', 'wb') as f:  
    pickle.dump([G_main, adj_mx, edges, G], f)

In [ ]:
# max_speed_dict = {}
# length_dict = {}
# node_size_dict = {}
# node_degree = {}

# for node in sample.values: # Loop through the list, one row at a time
#     length_dict[node[0]]=node[-1]
    
# for node in sample.values: # Loop through the list, one row at a time
#     node_size_dict[node[0]]=node[-1]/20
    
# for node in sample.values: # Loop through the list, one row at a time
#     max_speed_dict[node[0]]= node[-2]

# node_degree = dict(G.degree)

# nx.set_node_attributes(G, length_dict, 'length')
# nx.set_node_attributes(G, node_size_dict, 'size')
# nx.set_node_attributes(G, max_speed_dict, 'max_speed')
# nx.set_node_attributes(G, node_degree, 'node_degree')

In [ ]:
# import numpy as np
# import pandas as pd
# import holoviews as hv
# import networkx as nx
# from holoviews import opts

# hv.extension('bokeh')

# defaults = dict(width=1000, height=800)
# hv.opts.defaults(
#     opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))


In [ ]:
# kwargs = dict(width=1000, height=800, xaxis=None, yaxis=None)
# opts.defaults(opts.Nodes(**kwargs), opts.Graph(**kwargs))

# colors = ['#000000']+hv.Palette('Spectral').values
# g = hv.Graph.from_networkx(G, nx.layout.fruchterman_reingold_layout)
# labels = hv.Labels(g.nodes, ['x', 'y'], 'pos')

# g.opts(cmap=colors, pos= 'pos')

In [ ]:
# renderer = hv.renderer('bokeh')

# # Convert to bokeh figure then save using bokeh
# plot = renderer.get_plot(g).state

# from bokeh.io import output_file, save, show
# save(plot, 'graph.html')